<a href="https://colab.research.google.com/github/derekphilipau/machinelearningforartists/blob/main/stylegan2_ada_pytorch_yue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Pokemon with StyleGAN

In this tutorial, we only want to familiarize ourselves with running StyleGAN (in this case, the newest version) and seeing how training progresses.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install Stylegan2-ada-pytorch Prerequisites

In [ ]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [ ]:
torch.cuda.device_count()

In [ ]:
import torchvision

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## Get the StyleGAN code

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
mkdir /content/stylegan2-ada-pytorch/datasets

## Get Pokemon dataset of images

In [ ]:
cd /content/stylegan2-ada-pytorch/datasets

In [ ]:
!wget https://github.com/derekphilipau/machinelearningforartists/raw/main/docs/src/week2/pokemon256.zip

In [ ]:
!unzip pokemon256.zip

In [ ]:
!rm -rf __MACOSX

In [ ]:
!rm pokemon256.zip

In [ ]:
cd /content/stylegan2-ada-pytorch/

## Prepare Pokemon dataset for use by StyleGAN

In [ ]:
!python dataset_tool.py --source=./datasets/pokemon256 --dest=./datasets/pokemondataset.zip

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  The Pokemon training data will be stored in **MachineLearningForArtists/Pokemon**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/Pokemon

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/Pokemon --data=./datasets/pokemondataset.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=10 --metrics=none


## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/Pokemon

Each training run is stored in a separate directory.  The initial training run is stored in `00000-pokemondataset...`.  If you run training twice, the second run will be stored in `00001-pokemondataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various Pokemon in this image. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Transfer Learning

Now, let's train the same dataset but use *transfer learning*.  We will use a StyleGAN2 model already pre-trained on Pokemon to train a new model that is trained against a Tamagotchi dataset.

### Get the Tomagotchi dataset

In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets

In [ ]:
!wget https://github.com/derekphilipau/machinelearningforartists/raw/main/docs/src/week2/Tamagotchi256.zip

In [ ]:
!unzip Tamagotchi256.zip

In [ ]:
!rm -rf __MACOSX/

In [ ]:
%cd /content/stylegan2-ada-pytorch

### Prepare Tomagotchi dataset for use by StyleGAN

In [ ]:
!python dataset_tool.py --source=./datasets/Tamagotchi256 --dest=./datasets/tamagotchidataset.zip

### Get the pre-trained Pokemon model

In [ ]:
!gdown --id 122_Nu7p6ESMqBarxuAHCXhQuAaNCsh_a

### Train using the Pokemon model with the Tamagotchi dataset 

In [ ]:
!python train.py --resume=./stylegan2-ada-pytorch-pokemon-256px-11650.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/Tamagotchi --data=./datasets/tamagotchidataset.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=10 --metrics=none
